In [1]:
import pennylane as qml
import numpy as np
from pennylane.optimize import AdamOptimizer, NesterovMomentumOptimizer, AdagradOptimizer
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

def generate_parity_data(n_bits, num_samples):
    """
    Generate synthetic data for the n-bit parity problem.
    
    Parameters:
    - n_bits: Number of input bits.
    - num_samples: Number of samples to generate.
    
    Returns:
    - A DataFrame with n_bits input features and a target label (0 or 1).
    """
    X = np.random.randint(0, 2, size=(num_samples, n_bits))  # Generate random binary input
    y = np.sum(X, axis=1) % 2  # Compute parity (sum of bits mod 2)
    df = pd.DataFrame(X, columns=[f'bit_{i}' for i in range(n_bits)])
    df['label'] = y
    return df

# Generate datasets for 3-bit and 5-bit parity problems
train_3bit = generate_parity_data(n_bits=3, num_samples=1000)
test_3bit = generate_parity_data(n_bits=3, num_samples=200)

train_5bit = generate_parity_data(n_bits=5, num_samples=1000)
test_5bit = generate_parity_data(n_bits=5, num_samples=200)

In [4]:
# Initialize device
num_qubits = 4
dev = qml.device("default.qubit", wires=num_qubits)

# Define a layer of the variational quantum circuit
def layer(weights):
    for i in range(num_qubits):
        qml.Rot(*weights[i], wires=i)
    for i in range(num_qubits):
        qml.CNOT(wires=[i, (i+1) % num_qubits])

# Data re-uploading in state preparation
def state_preparation(x):
    for i in range(num_qubits):
        qml.RX(x[i % len(x)], wires=i)


In [5]:
@qml.qnode(dev, interface='autograd')
def circuit(weights, x):
    state_preparation(x)
    for w in weights:
        layer(w)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [6]:
def square_loss(labels, predictions):
    return qml.numpy.mean((labels - predictions) ** 2)

def accuracy(labels, predictions):
    return qml.numpy.mean(labels == qml.numpy.sign(predictions))

def cost(weights, bias, X, Y):
    predictions = qml.numpy.array([variational_classifier(weights, bias, x) for x in X])
    return square_loss(Y, predictions)

In [14]:
def generate_quantum_data(num_features, num_samples):
    X = np.random.uniform(0, 2 * np.pi, size=(num_samples, num_features))  # Generate angles in [0, 2π]
    y = np.random.choice([-1, 1], size=num_samples)  # Binary labels for classification
    return X, y

# Set parameters matching the quantum circuit (4 qubits)
num_qubits = 4
num_samples = 500

# Generate synthetic quantum data
X_quantum, y_quantum = generate_quantum_data(num_qubits, num_samples)

# Convert to DataFrame for easier inspection
df = pd.DataFrame(X_quantum, columns=[f'feature_{i}' for i in range(num_qubits)])
df['label'] = y_quantum

X_train = np.array(df.drop(columns = 'label'))
y_train = np.array(df['label'])

In [ ]:
np.random.seed(0)
num_layers = 4
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3)
weights_init = qml.numpy.array(weights_init, requires_grad=True)
bias_init = qml.numpy.array(0.0, requires_grad=True)

opt = NesterovMomentumOptimizer(0.1)
batch_size = 10
num_iterations = 100

weights = weights_init
bias = bias_init

costs = []

for it in range(num_iterations):
    # Random batch sampling
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = y_train[batch_index]

    weights, bias = opt.step(cost, weights, bias, X=qml.numpy.array(X_batch), Y=qml.numpy.array(Y_batch))

    # Accuracy and cost on the training set
    predictions = [variational_classifier(weights, bias, x) for x in X_train]
    acc_train = accuracy(y_train, qml.numpy.sign(predictions))
    cost_train = cost(weights, bias, X_train, y_train)
    costs.append(cost_train)

    if (it + 1) % 10 == 0:
        print(f"Iteration: {it + 1} | Cost: {cost_train:.4f} | Training Accuracy: {acc_train:.4f}")

# Evaluation on the test set
predictions_test = [variational_classifier(weights, bias, x) for x in X_test]
acc_test = accuracy(y_test, qml.numpy.sign(predictions_test))

print(f"Test Accuracy: {acc_test:.4f}")

# Plotting the cost over iterations
plt.plot(costs)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.title("Cost over Iterations")
plt.show()

Iteration: 10 | Cost: 1.2538 | Training Accuracy: 0.5120
Iteration: 20 | Cost: 1.4046 | Training Accuracy: 0.5120
